In [7]:
#Activity 5. Sentiment Analysis on a RealLife Data-Set
#1. Import DATA using pandas read_csv
import matplotlib.pyplot as plt
import pandas as pd
import os
here = os.path.dirname(__file__) if "__file__" in locals() else "."
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin'

# files = [("amazon", os.path.join(here, "./Sentiment_labelled_sentences//amazon_cells_labelled.txt")),
#          ("imdb", os.path.join(here, "./Sentiment_labelled_sentences//imdb_labelled.txt")),
#          ("yelp", os.path.join(here, "./Sentiment_labelled_sentences//yelp_labelled.txt"))]

files = [("amazon", os.path.join(here, "./Sentiment_labelled_sentences//yelp_labelled.txt"))]

# df = pd.read_csv('./Sentiment_labelled_sentences/amazon_cells_labelled.txt', delimiter="\t",sep=" ", header=None)
# df.columns = ["Sentence", "Label"]
dfs = [] # define an empty array
for provider, name in files:
    df = pd.read_csv(name, sep="\t")
    df.columns = ["Sentence", "Label"]
    df["provider"] = provider
    dfs.append(df)
#dfs now has 3 elements which are 3 arrays in 3 columns
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
Data = pd.concat(dfs,axis = 0) # join 3 framses 
# print(Data.head(10))
# print("shape", Data.shape)
import re
#1.2 Create a func to preproces the reviews (clean and prepare data)
def clean_comment(text):
    #Strip HTML tags
    text = re.sub('<[^<]+?>', ' ', text)
    #Strip escaped quotes
    text = text.replace('\\"', '')
    # text = re.sub('//"', '', text)
    #Strip quotes
    text = text.replace('"', '')
    # text = re.sub('"', '', text)
    return text
Data = Data.dropna()
Data["cleaned_cmts"] = Data["Sentence"].apply(clean_comment) 

####? Should we add Lower casing and Removing stop_words
# print(Data.Label.value_counts()) , count positive and negative comments

#1.3 Define variables 
Batch_size = 32
Epochs = 20
Maxlen = 30  # Pre-define Vocabulary size, this size affects heavily on the training time
Embedded_Dim = 50 # Embedding Dimension, output vector
Num_filters = 64
Kernel_Size = 5 

#2. Split Data to Train and Test using scikit-learn'S train_test_split 
import numpy as np
from sklearn.model_selection import  train_test_split
X_train, X_test, y_train, y_test = train_test_split(Data["cleaned_cmts"],Data["Label"], test_size=0.30, random_state = 1000) # random_state: keep the data the same evertime it runs

# print(max_len)
# for x in range(len(X_train)): 
#     print("element ",x,"has len of ",len(X_train[x])) 

#3. Tokenize using Keras's tokenizer 
import keras
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=100000) # num_words: the maximum number of words to keep, Bigger -> Better
# Build Vocab based on training data ? < What if Test data has the words that not in Training Data 
tokenizer.fit_on_texts(X_train)


#4.Use texts_to_sequences: Transforms each text in texts to a sequence of integers  
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
Vocab_size =  len(tokenizer.word_index) + 1 # The Vocab size has an additional 1, due to 0 index

#5.  Ensure that all the sequencese have the same length by padding them, using Keras's pad_sequences function
from keras.preprocessing.sequence import pad_sequences
X_train_padded = pad_sequences(X_train, padding = 'post', truncating = 'post', maxlen = Maxlen) 
# 
X_test_padded = pad_sequences(X_test, padding = 'post', truncating = 'post', maxlen = Maxlen)
 # , maxlen=20)
# for count in range(6):
#  print(X_test_padded[count])


#6.  Define Model with minimum 1 ConV, 1 Fully Connected, using 'sigmoid' as activation function, calculate the loss to cross-entropy loss
from keras.models import Sequential
from keras import layers
from keras.utils import plot_model

MODEL = Sequential()
MODEL.add(layers.Embedding(input_dim = Vocab_size, output_dim= Embedded_Dim, input_length = Maxlen)) # traineble = false, no update to learned data. 
MODEL.add(layers.Conv1D(Num_filters, Kernel_Size, activation='relu'))
MODEL.add(layers.GlobalMaxPooling1D())
MODEL.add(layers.Dense(units = 10, activation='relu'))
MODEL.add(layers.Dense(units = 1, activation = 'sigmoid'))
MODEL.compile(loss ='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
MODEL.summary()

History = MODEL.fit(X_train_padded,y_train, epochs = Epochs, batch_size = Batch_size, verbose = 0, validation_data =(X_test_padded,y_test))

Score = MODEL.evaluate(X_train_padded,y_train, batch_size = Batch_size, verbose = 0)
print("Train Loss: {:.4f}".format(Score[0]))
print("Train Accuracy:{:.4f}".format(Score[1]))

Score1 = MODEL.evaluate(X_test_padded,y_test, batch_size = Batch_size, verbose = 0)
print("Test Loss: {:.4f}".format(Score1[0]))
print("Test Accuracy: {:.4f}".format(Score1[1]))




Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 50)            84850     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 26, 64)            16064     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 101,575
Trainable params: 101,575
Non-trainable params: 0
_________________________________________________________________
Train Loss: 0.0009
Train Accuracy:1.0000
Test Loss: 0.6991
Test Accuracy: 0.7767


In [12]:
# Bonus 1: example to understand Tokenizer and Text_to_sequences method of Keras 
import keras
from keras.preprocessing.text import Tokenizer
tok = Tokenizer()
train_text = ["this girl is looking beautiful!!"] 
test_text = ["this girl is not really looking beautiful"]
test_text2 = ["this girl is beautiful"]
test_text3 = "bad girl"
test_text4 = "This girl is not really looking beautiful and nice"
# This is for updating internal vocabulary based on a list of texts
tok.fit_on_texts(train_text) 
print("word_index : ",tok.word_index)

#texts_to_sequences Transforms each text in texts to a sequence of integers.
tr1 = tok.texts_to_sequences(train_text) 
sequences1 = tok.texts_to_sequences(test_text)
sequences2 = tok.texts_to_sequences(test_text2)

sequences34 = tok.texts_to_sequences([test_text3 ,test_text4])
# t4 = tok.texts_to_sequences(test_text4)
print(tr1) # Vocabulary was built with 5 words 
print(sequences1)  # it only recognized the words in Vocab
print(sequences2)  # all the word are already in the Vocab 
print(sequences34) # "Bad" was not recognized 
# print(t4) # It can not recognized the words which are not in the Vocab 




word_index :  {'this': 1, 'girl': 2, 'is': 3, 'looking': 4, 'beautiful': 5}
[[1, 2, 3, 4, 5]]
[[1, 2, 3, 4, 5]]
[[1, 2, 3, 5]]
[[2], [1, 2, 3, 4, 5]]


In [11]:

    print(*X_train_padded, sep = "\n")
    print(*X_test_padded, sep = "\n")

  0   0   0   0   0   0]
[  13    6    1 1200   11    2  354    7    5 1655    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[ 80  20 142 905  66   1   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[26 28  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0]
[  5 218   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[ 13   5  30 117   5  15   6 173  33   6 256 333  14   5 213   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[  4  43 108 214  32  77  26  12  56 424   2 285  82  85  26 178   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[   4 1531    1  525   30   87   13  833  148    4  332    1  121   87
    5  178  359    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[1662   11   22   76   11    1  104  217  885    1    0    0    0    0
    0    0 